# Global full land (snow+soil+canopy) run

The code sets up the ClimaLand land model  on a spherical domain,
forcing with ERA5 data, but does not actually run the simulation.
To run the simulation, we strongly recommend using a GPU.

First we import a lot of packages:

In [ ]:
ENV["JULIA_PKG_PRECOMPILE_AUTO"] = 0
required_pkgs = ["CairoMakie", "ClimaAnalysis", "GeoMakie", "Printf", "StatsBase", "ClimaComms", "ClimaParams", "Dates", "ClimaUtilities", "Interpolations", "ClimaDiagnostics", "ClimaLand"]
import Pkg
Pkg.Registry.update()
Pkg.add(required_pkgs)

In [ ]:
import ClimaComms
ClimaComms.@import_required_backends
import ClimaParams as CP
using Dates
using ClimaUtilities
import ClimaUtilities.TimeVaryingInputs:
    TimeVaryingInput, LinearInterpolation, PeriodicCalendar
import Interpolations
using ClimaDiagnostics
import ClimaLand
import ClimaLand.Parameters as LP
import ClimaLand.Simulations: LandSimulation, solve!

using CairoMakie, ClimaAnalysis, GeoMakie, Printf, StatsBase
import ClimaLand.LandSimVis as LandSimVis

In [ ]:
pkgversion(ClimaLand)

Set the simulation float type, determine the
context (MPI or on a single node), and device type (CPU or GPU).
Create a default output directory for diagnostics.

In [ ]:
const FT = Float64;
context = ClimaComms.context()
ClimaComms.init(context)
device = ClimaComms.device()
device_suffix = device isa ClimaComms.CPUSingleThreaded ? "cpu" : "gpu"
root_path = "land_longrun_$(device_suffix)"
diagnostics_outdir = joinpath(root_path, "global_diagnostics")
outdir =
    ClimaUtilities.OutputPathGenerator.generate_output_path(diagnostics_outdir);
earth_param_set = LP.LandParameters(FT);

Set timestep, start_date, stop_date:

In [ ]:
Δt = 450.0
start_date = DateTime(2008)
stop_date = DateTime(2009);

Create the domain:

In [ ]:
nelements = (101, 15)
domain = ClimaLand.Domains.global_domain(FT; context, nelements);

Low-resolution forcing data from ERA5 is used here,
but high-resolution should be used for production runs.

In [ ]:
era5_ncdata_path = ClimaLand.Artifacts.era5_land_forcing_data2008_path(;
    context,
    lowres = true,
)
forcing = ClimaLand.prescribed_forcing_era5(
    era5_ncdata_path,
    domain.space.surface,
    start_date,
    earth_param_set,
    FT;
    max_wind_speed = 25.0,
    time_interpolation_method = LinearInterpolation(PeriodicCalendar()),
    regridder_type = :InterpolationsRegridder,
);

MODIS LAI is prescribed for the canopy model:

In [ ]:
modis_lai_ncdata_path = ClimaLand.Artifacts.modis_lai_multiyear_paths(;
    context,
    start_date,
    end_date = stop_date,
)
LAI = ClimaLand.prescribed_lai_modis(
    modis_lai_ncdata_path,
    domain.space.surface,
    start_date;
    time_interpolation_method = LinearInterpolation(),
);

In [ ]:
subsurface_space = domain.space.subsurface
nc_writer =
   ClimaDiagnostics.Writers.NetCDFWriter(subsurface_space, outdir; start_date);
​​diagnostics = ClimaLand.default_diagnostics(
   model,
   start_date;
   output_writer = nc_writer,
   average_period = :daily, # can also be :hourly or :monthly
);


Make the model and simulation:

In [ ]:
model = ClimaLand.LandModel{FT}(forcing, LAI, earth_param_set, domain, Δt);
simulation = ClimaLand.Simulations.LandSimulation(
    start_date,
    stop_date,
    Δt,
    model;
    outdir,
    user_callbacks = (ClimaLand.ReportCallback(10000),)

);

Run the simulation, with a performance report output every 10,000 steps (every 5.2 days):

In [ ]:
ClimaLand.Simulations.solve!(simulation)

## Let's make some plots!

In [ ]:
LandSimVis.make_annual_timeseries(simulation; savedir = root_path)
LandSimVis.make_heatmaps(simulation;date = stop_date, savedir = root_path)
LandSimVis.make_leaderboard_plots(simulation, savedir = root_path)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*